In [ ]:
import pandas as pd 
import numpy as np 
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from sklearn.preprocessing import SplineTransformer, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
import holidays

from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from mlforecast import MLForecast
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

# Data analysis

In [ ]:
prediction_data = pd.read_parquet(Path.cwd().parent.joinpath("data", "public_test_data.parquet"))
prediction_data = prediction_data.set_index("Timestamp_Local")
prediction_data = prediction_data.sort_values(["Site", "Timestamp_Local"])

In [ ]:
prediction_data.info()

In [ ]:
training_data = pd.read_parquet(Path.cwd().parent.joinpath("data", "training_data.parquet"))
training_data = training_data.set_index("Timestamp_Local")
training_data = training_data.sort_values(["Site", "Timestamp_Local"])
# Add ground truth baseline power
training_data['Baseline_Power_kW'] = training_data['Building_Power_kW'] - training_data['Demand_Response_Capacity_kW']

original_cols = [
    "Site",
    "Building_Power_kW",
    "Demand_Response_Capacity_kW",
    "Demand_Response_Flag",
    "Dry_Bulb_Temperature_C",
    "Global_Horizontal_Radiation_W/m2",
]

In [ ]:
training_data.info()

## Raw data

In [ ]:
# for site, site_data in training_data.groupby("Site"):
#     data_cols = (
#             "Building_Power_kW",
#             "Demand_Response_Capacity_kW",
#             "Demand_Response_Flag",
#             "Dry_Bulb_Temperature_C",
#             "Global_Horizontal_Radiation_W/m2",
#     )
#     fig = make_subplots(
#         rows=5,
#         cols=1,
#         shared_xaxes=True,
#         vertical_spacing=0.02
#     )
#     for row, col in enumerate(data_cols, start=1):
#         fig.add_trace(
#             go.Scatter(
#                 x=site_data.index,
#                 y=site_data[col],
#                 name=col,
#                 mode="lines",
#             ),
#             row=row,
#             col=1
#         )
#     fig.update_layout(title_text=f"Site: {site}")
#     fig.show()

## Demand reduction response analysis

There is not always a capacity reduction / increase when the demand response is set.
This implies that the building is not always able to deliver flexibility when requested.

In [ ]:
# # Is there always a demand response when the flag is set?
# training_data['Demand_Response_Delivered'] = training_data['Demand_Response_Capacity_kW'].ne(0)
# (training_data.groupby(['Site', 'Demand_Response_Flag'])['Demand_Response_Delivered'].sum() / training_data.groupby(['Site', 'Demand_Response_Flag'])['Demand_Response_Delivered'].count()).reset_index()

# training_data['Demand_Response_Direction'] = np.where(
#     np.sign(training_data['Demand_Response_Capacity_kW']).astype(int) == training_data['Demand_Response_Flag'],
#     'Correct response',
#     np.where(
#         training_data['Demand_Response_Capacity_kW'] == 0,
#         'No response',
#         'Inverse response'
#     )
# )
# demand_response_performance = (training_data.groupby(['Site', 'Demand_Response_Flag'])['Demand_Response_Direction'].value_counts() / training_data.groupby(['Site', 'Demand_Response_Flag'])['Demand_Response_Direction'].count()).reset_index()
# fig = make_subplots(rows=3, cols=1, subplot_titles=demand_response_performance['Site'].unique().tolist(), shared_xaxes=True)
# colors = {
#     'No response': 'gray',
#     'Correct response': '#bdf0a8',
#     'Inverse response': '#f09999'
# }
# for direction, performance in demand_response_performance.groupby('Demand_Response_Direction'):
#     first_legend=True
#     for row, (site, site_performance) in enumerate(performance.groupby('Site'), start=1):
#         fig.add_trace(go.Bar(
#             x=site_performance['Demand_Response_Flag'].astype(str),
#             y=site_performance[0],
#             name=direction,
#             legendgroup=direction,
#             text=site_performance[0].round(2),
#             textposition='auto',
#             marker_color=colors.get(direction, 'blue'),
#             showlegend=first_legend
#         ),
#         row=row, col=1)
#         first_legend=False
# fig.update_layout(
#     barmode='stack',
#     height=600,
# )
# fig.show()

## Stationarity

In [ ]:
# for site, site_data in training_data.groupby("Site"):
#     result = adfuller(site_data['Building_Power_kW'])

#     print(f'{site=}')
#     print(f'\tADF Statistic: {result[0]:.4f}')
#     print(f'\tp-value: {result[1]:.4f}')
#     print('\tCritical Values:')

#     for key, value in result[4].items():
#         print(f'\t\t{key}: {value:.3f}')
    
#     if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
#         print(f'\t\u001b[32m*** The time series is stationary (reject H0) ✅ ***\u001b[0m')
#     else:
#         print(f'\t\x1b[31m*** The time series is non-stationary (fail to reject H0) ❌ ***\x1b[0m')

# # Visualizations
# plt.rcParams.update({'font.size': 8})
# lag_acf = 15
# lag_pacf = 15
# height = 2
# width = 8

# for site, site_data in training_data.groupby("Site"):
#     print(f'{site=}')
#     f, ax = plt.subplots(nrows=2, ncols=1, figsize=(width, 2*height))
#     plot_acf(site_data['Building_Power_kW'],lags=lag_acf, ax=ax[0])
#     plot_pacf(site_data['Building_Power_kW'],lags=lag_pacf, ax=ax[1], method='ols')

#     plt.tight_layout()
#     plt.show()

# def plot_components(result, site):
#     df = pd.concat([result.observed, result.trend, result.seasonal, result.resid], axis=1)
#     df = df.rename(columns={0:'Original Data', 'season':'seasonal','observed':'Original Data'})
#     components = df.columns
#     rows = len(components)
#     fig = make_subplots(rows=rows, cols=1, shared_xaxes=True, subplot_titles = [i for i in components])

#     # Plot original data
#     for i, col in enumerate(components):
#         fig.add_trace(go.Scatter(x=df.index, y=df[col], mode='lines', name=col), row=i+1, col=1)

#     # Update layout
#     fig.update_layout(
#         title=f'Time Series Decomposition for {site}',
#         xaxis_title='Time',
#         height=900
#     )

#     fig.show()

# period = int((60/15)*24*7)
# for site, site_data in training_data.groupby("Site"):
#     stl = STL(site_data['Building_Power_kW'], period=period)
#     result_stl = stl.fit()

#     # Plot the results
#     plot_components(result_stl, site)

There is not much seasonality in the training data. For Site A and Site B there is a more distinct turn on and turn off event.

# Feature engineering

In [ ]:
def add_datetime_features(data: pd.DataFrame) -> pd.DataFrame:
    # 1: summer, 2: autumn, 3: winter, 4: spring
    season_lookup = {
        1: 1,
        2: 1,
        3: 2,
        4: 2,
        5: 2,
        6: 3,
        7: 3,
        8: 3,
        9: 4,
        10: 4,
        11: 4,
        12: 1
    }

    data = data[[c for c in original_cols + ['Baseline_Power_kW'] if c in data.columns]]
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['hour'] = data.index.hour
    data['minute'] = data.index.minute
    data['day_of_week'] = data.index.dayofweek
    data['working_day'] = np.where(data['day_of_week'].isin([0,1,2,3,4]), True, False)
    data['holiday'] = data.index.normalize().isin(holidays.country_holidays('AU'))
    data['season'] = data['month'].map(season_lookup)
    data['working_hour'] = np.where((data['hour'] >= 9) & (data['hour'] <= 17) & (data['working_day']), True, False)

    # Circular features
    def periodic_spline_transformer(period, n_splines=None, degree=3):
        if n_splines is None:
            n_splines = period
        n_knots = n_splines + 1  # periodic and include_bias is True
        return SplineTransformer(
            degree=degree,
            n_knots=n_knots,
            knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
            extrapolation="periodic",
            include_bias=True,
        )
    
    return data

def add_lag_features(data: pd.DataFrame) -> pd.DataFrame:
    data['power_lag_15_min'] = data.groupby('Site')['Building_Power_kW'].shift(1)
    data['power_lag_30_min'] = data.groupby('Site')['Building_Power_kW'].shift(2)
    data['power_lag_60_min'] = data.groupby('Site')['Building_Power_kW'].shift(4)
    data['power_lag_1_day'] = data.groupby('Site')['Building_Power_kW'].shift(96)

    return data

def add_hvac_flag(data: pd.DataFrame) -> pd.DataFrame:
    # HVAC flag
    data['HVAC_on'] = data.groupby(['Site', pd.Grouper(freq='1W')])['Building_Power_kW'].transform(
        lambda x: x >= 0.5 * x.mean()
    )
    return data

def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    data = add_datetime_features(data)
    data = add_lag_features(data)
    data = add_hvac_flag(data)
    return data

# Baseline model

In [ ]:
training_data = feature_engineering(training_data)

In [ ]:
train_test_split = 0.75
test_data = []
train_data = []
for site, site_data in training_data.groupby("Site"):
    complete_site_data = site_data.dropna(how='any')
    split_index = int(len(complete_site_data) * train_test_split)
    train_data.append(complete_site_data.iloc[:split_index])
    test_data.append(complete_site_data.iloc[split_index:])
train_data = pd.concat(train_data)
test_data = pd.concat(test_data)

train_data = train_data[
    [c for c in train_data.columns if c not in [
            'Demand_Response_Capacity_kW',
        ]
    ]
].reset_index()
test_data = test_data[
    [c for c in test_data.columns if c not in [
            'Demand_Response_Capacity_kW',
        ]
    ]
].reset_index()

In [ ]:
# fig = make_subplots(specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]], rows=3, cols=1, subplot_titles=training_data['Site'].unique().tolist())
# for row, (site, site_data) in enumerate(training_data.groupby("Site"), start=1):
#     fig.add_trace(go.Scatter(x=site_data.index, y=site_data['Building_Power_kW'], mode='lines', name='Building Power (kW)', line_color=px.colors.qualitative.Plotly[0]), secondary_y=False, row=row, col=1)
#     fig.add_trace(go.Scatter(x=site_data.index, y=site_data['HVAC_on'].astype(int), mode='lines', name='HVAC', line_color='gray'), secondary_y=True, row=row, col=1)
#     fig.update_yaxes(range=[-0.1, 1.1], secondary_y=True, row=row, col=1)
# fig.update_layout(height=900)
# fig.show()

In [ ]:
def plot_predictions(data, cols):
  fig = make_subplots(rows=len(data['Site'].unique()), cols=1, subplot_titles=data['Site'].unique().tolist())

  for row, (_, site_data) in enumerate(data.groupby('Site'), start=1):
      for i, col in enumerate(cols):
        fig.add_trace(go.Scatter(x=site_data['Timestamp_Local'], y=site_data[col], mode='lines', name=col, line_color=px.colors.qualitative.Plotly[i]), row=row, col=1)
  fig.update_layout(height=row*300)
  fig.show()

In [ ]:
train_data

In [ ]:
model = lgb.LGBMRegressor(
    random_state=420,
    verbose=-1
)

model.fit(
    X=train_data.drop(columns=['Site', 'Timestamp_Local', 'Baseline_Power_kW']), 
    y=train_data['Baseline_Power_kW']
)

In [ ]:
predicted_data = []
for site, site_data in test_data.groupby('Site'):
    x = model.predict(site_data.drop(columns=['Site', 'Timestamp_Local', 'Baseline_Power_kW']))
    predicted_data.append(pd.DataFrame(data={'prediction' : x, 'Site' : site, 'Timestamp_Local' : site_data['Timestamp_Local']}))
predicted_data = pd.concat(predicted_data)
predicted_data = predicted_data.set_index(['Site', 'Timestamp_Local'])

predicted_data = pd.merge(left=predicted_data, right=test_data.set_index(['Site', 'Timestamp_Local'])[['Baseline_Power_kW']].rename(columns={'Baseline_Power_kW' : 'actual'}), left_index=True, right_index=True)
predicted_data = predicted_data.reset_index()

In [ ]:
mae = mean_absolute_error(predicted_data['actual'], predicted_data['prediction'])
mape = mean_absolute_percentage_error(predicted_data['actual'], predicted_data['prediction'])

print(f'MAE: {mae:,.4f} | MAPE: {mape:.4%}')

In [ ]:
predicted_data

In [ ]:
plot_predictions(predicted_data, cols=['actual', 'prediction'])

# Prediction

In [ ]:
prediction_data = feature_engineering(prediction_data)
prediction_data = prediction_data.reset_index()

In [ ]:
predicted_data = []
for site, site_data in prediction_data.groupby('Site'):
    x = model.predict(site_data.drop(columns=['Site', 'Timestamp_Local']))
    predicted_data.append(pd.DataFrame(data={'prediction' : x, 'Site' : site, 'Timestamp_Local' : site_data['Timestamp_Local']}))
predicted_data = pd.concat(predicted_data)
predicted_data = predicted_data.set_index(['Site', 'Timestamp_Local'])

predicted_data = pd.merge(left=predicted_data, right=prediction_data.set_index(['Site', 'Timestamp_Local']), left_index=True, right_index=True)
predicted_data = predicted_data.reset_index()

predicted_data['Demand_Response_Capacity_kW'] = predicted_data['Building_Power_kW'] - predicted_data['prediction']
# If the demand response flag is set to 0, the capacity must also be 0
predicted_data['Demand_Response_Capacity_kW'] = np.where(predicted_data['Demand_Response_Flag'] == 0, 0, predicted_data['Demand_Response_Capacity_kW'])
# If demand response capacity is less than 10% of the building power, set it to 0
predicted_data['Demand_Response_Capacity_kW'] = np.where(np.abs(predicted_data['Demand_Response_Capacity_kW']) < 0.1 * predicted_data['Building_Power_kW'], 0, predicted_data['Demand_Response_Capacity_kW'])
predicted_data['Demand_Response_Flag'] = np.where(predicted_data['Demand_Response_Flag'].isna(),
    np.where(predicted_data['Demand_Response_Capacity_kW'] > 0, 1, np.where(predicted_data['Demand_Response_Capacity_kW'] < 0, -1, 0)),
    predicted_data['Demand_Response_Flag']
)
predicted_data['Demand_Response_Flag'] = predicted_data['Demand_Response_Flag'].astype(int)

predicted_data[['Site', 'Timestamp_Local', 'Demand_Response_Flag', 'Demand_Response_Capacity_kW']].to_csv(Path.cwd().parent.joinpath("data", "submission.csv"), index=False)

In [ ]:
plot_predictions(predicted_data, cols=['Building_Power_kW', 'prediction', 'Demand_Response_Flag', 'Demand_Response_Capacity_kW'])